## AlekseevDP(DSU-4,DLL-15)_HW#7_Рекуррентные НС (Часть 3). Исправлен LSTM

Задание:
- Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
- Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
- Попробуйте добавить +1 рекуррентный слой в encoder и decoder
- Попробуйте заменить GRU-ячейки на LSTM-ячейки

Оцените качество во всех случаях



In [1]:
%matplotlib inline

In [2]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING = "1"

In [ ]:
# !wget https://www.manythings.org/anki/rus-eng.zip
# !unzip rus-eng.zip

## Файл https://www.manythings.org/anki/rus-eng.zip потребовал предобработки в программе Anki, а именно - удаления колонки со служебной информацией (номер  карточки перевода, автор и т.п.), иначе при загрузке корпуса возникала ошибка.Обработанный файл: eng-rus_prepared_full.txt (для быстрого примера см. в репозитории Github файл 'eng-rus_prepared_example.txt')

In [3]:
# переместим 'eng-rus_prepared_full.txt' в каталог '/data' и переименуем в 'eng-rus.txt'
!mkdir '/content/data'
!mv eng-rus_prepared_full.txt '/content/data/eng-rus.txt'

In [4]:
!head -50 data/eng-rus.txt

Go.	Марш!
Hi.	Здравствуйте.
Run!	Беги!
Run.	Беги!
Who?	Кто?
Wow!	Вот это да!
Duck!	Пригнись!
Fire!	Огонь!
Help!	Помогите!
Hide.	Прячься.
Jump!	Прыгай!
Jump.	Прыгай!
Stay.	Оставайся.
Stop!	Стой!
Wait!	Подожди!
Wait.	Ждите.
Do it.	Сделай это.
Go on.	Продолжай.
Hello!	Здравствуйте.
Hurry!	Поспешите.
I ran.	Я бежал.
I see.	Понимаю.
I try.	Я пытаюсь.
I won!	Я победил!
Oh no!	О нет!
Relax.	Расслабьтесь.
Shoot!	Стреляй!
Smile.	Улыбочка.
Sorry?	Извините?
Attack!	В атаку!
Buy it.	Купите её.
Cheers!	За ваше здоровье!
Eat it.	Съешь это.
Eat up.	Доедай.
Freeze!	Ни с места!
Get up.	Вставай.
Go now.	А теперь уходи.
Got it!	Понял!
Got it?	Понял?
He ran.	Он бежал.
Hop in.	Залезай.
Hug me.	Обними меня.
I fell.	Я упал.
I knit.	Я вяжу.
I know.	Я знаю.
I left.	Я ушёл.
I lied.	Я солгал.
I lost.	Я проиграл.
I paid.	Я заплатил.
I pass.	Я пас.


In [5]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

## Через библиотеку Unicodedata не удалось "забороть" кодировку предложений на русском языке, хотя кодировка UTF-8 включает в себя в т.ч. русские буквы (ошибка не возникала, но русскоязычная часть корпуса получалась всегда пустой - 0 фраз). Поэтому нашел альтернативный вариант - русские фразы переводить в транслит с использованием другой библиотеки (Unidecode):

In [6]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.7 MB/s 


In [7]:
from unidecode import unidecode

In [8]:
def unicodeToAscii(s):
    return ''.join(
        # c for c in unicodedata.normalize('NFD', s)  # Turn a Unicode string to plain ASCII, 
        # if unicodedata.category(c) != 'Mn'          # thanks to http://stackoverflow.com/a/518232/2809427
        c for c in unidecode(s)
)

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [9]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

## На "типовых" префиксах начала предложений (из туториала/лекции) модель Seq2seq на 1-слойных GRU-энкодере/декодере показывала хорошую точность предсказаний (лосс ниже 1.0), т.к. корпус был небольшой и предложения короткие. Поэтому с целью эксперимента задача была усложнена: увеличена длина предложения до 100 символов, а также добавлены новые префиксы начала предложений. Итоговый корпус для обучения модели составил 29743 фразы (см.ниже).

In [10]:
MAX_LENGTH = 100

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re",
    "i can", "i can t",
    "that is", "that s",
    "we will", "we ll",
    "i was", "he was", "she was",
    "do you", "are you",
    "i have", "i ve", "i had", "i d",
    "what is", "what s", "what was",
    "what do", "what did",
    "we have", "we ve",
    "i do", "i don t",
    "did", "i ll",
    "there is", "there s",
    "how", "when"
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [11]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 746410 sentence pairs
Trimmed to 29743 sentence pairs
Counting words...
Counted words:
rus 13446
eng 6084
['vy delaete eto chashche raza v nedeliu ?', 'do you do that more than once a week ?']


In [13]:
print(random.choice(pairs))

['ia vizhu vershinu gory .', 'i can see the top of the mountain .']


The Encoder (1-layer GRU)
-----------





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder (1-layer GRU)
-----------




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Определим функции преобразования данных и обучения модели.

In [14]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [15]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [18]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Обучим  модель Seq2seq с использованием 1-слойных GRU для энкодера&декодера.С целью повышения качества предсказаний количество эпох увеличим вдвое (с 75000 до 150000)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 150000, print_every=5000) 

1m 0s (- 29m 0s) (5000 3%) 3.7474
1m 51s (- 26m 4s) (10000 6%) 3.2412
2m 44s (- 24m 43s) (15000 10%) 2.9723
3m 37s (- 23m 33s) (20000 13%) 2.7699
4m 30s (- 22m 31s) (25000 16%) 2.6018
5m 22s (- 21m 31s) (30000 20%) 2.4844
6m 15s (- 20m 33s) (35000 23%) 2.3738
7m 8s (- 19m 37s) (40000 26%) 2.2868
8m 1s (- 18m 42s) (45000 30%) 2.2007
8m 54s (- 17m 48s) (50000 33%) 2.1600
9m 46s (- 16m 53s) (55000 36%) 2.0674
10m 39s (- 15m 59s) (60000 40%) 1.9800
11m 33s (- 15m 6s) (65000 43%) 1.9497
12m 26s (- 14m 12s) (70000 46%) 1.9243
13m 19s (- 13m 19s) (75000 50%) 1.8637
14m 13s (- 12m 26s) (80000 53%) 1.8398
15m 6s (- 11m 32s) (85000 56%) 1.7801
15m 58s (- 10m 39s) (90000 60%) 1.7514
16m 51s (- 9m 45s) (95000 63%) 1.7061
17m 44s (- 8m 52s) (100000 66%) 1.7122
18m 37s (- 7m 58s) (105000 70%) 1.6542
19m 30s (- 7m 5s) (110000 73%) 1.6396
20m 23s (- 6m 12s) (115000 76%) 1.5780
21m 17s (- 5m 19s) (120000 80%) 1.5571
22m 9s (- 4m 25s) (125000 83%) 1.5464
23m 2s (- 3m 32s) (130000 86%) 1.4969
23m 55s (- 

In [ ]:
evaluateRandomly(encoder1, decoder1)

> ona vsio eshchio ego liubit .
= she still loves him .
< she still loves him . <EOS>

> ona s nim surova .
= she is hard on him .
< she is hard on him . <EOS>

> ty kogo nibud tam videl ?
= did you see anybody there ?
< did you see anyone there ? <EOS>

> kak liudi tochili karandashi do izobreteniia tochilki ?
= how did people sharpen their pencils before the invention of the pencil sharpener ?
< how did the get get the of get ? <EOS>

> vy izuchaete frantsuzskii ?
= do you study french ?
< do you study french ? <EOS>

> mne nado znat seichas .
= i have to know now .
< i have to know now . <EOS>

> ia sdelal eto dlia nas .
= i did that for us .
< i did it for for . <EOS>

> on istekal krov iu ot ran .
= he was bleeding from his wounds .
< he is from from from . . <EOS>

> kogda ty tuda doberesh sia ?
= when are you going to get there ?
< when do you get there ? <EOS>

> ia sobiralsia zadat tebe tot zhe vopros .
= i was about to ask you the same question .
< i was the to ask you to ask

## Мы видим, что визуально качество перевода не очень хорошее и лосс гораздо больше 1.0. Возможно, при увеличении количества эпох ещё в несколько раз качество перевода удастся существенно улучшить, но эта операция очень ресурсоемкая: на GPU Google Colab (был выделен Tesla T4) обучение модели заняло ок.30 минут. Попробуем добавить по одному рекуррентному слою GRU в Encoder и Decoder (просто переопределим классы).

In [ ]:
## 2-layers GRU @Encoder
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        # return torch.zeros(1, 1, self.hidden_size, device=device)
        return torch.zeros(2, 1, self.hidden_size, device=device)

In [ ]:
## 2-layers GRU @Decoder
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        # return torch.zeros(1, 1, self.hidden_size, device=device)
        return torch.zeros(2, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 256
num_layers = 2
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words, num_layers).to(device)

trainIters(encoder1, decoder1, 150000, print_every=5000)

1m 10s (- 34m 18s) (5000 3%) 3.7949
2m 12s (- 31m 1s) (10000 6%) 3.2709
3m 14s (- 29m 12s) (15000 10%) 3.0179
4m 17s (- 27m 51s) (20000 13%) 2.8252
5m 20s (- 26m 41s) (25000 16%) 2.6823
6m 23s (- 25m 32s) (30000 20%) 2.5338
7m 25s (- 24m 25s) (35000 23%) 2.4270
8m 29s (- 23m 19s) (40000 26%) 2.3474
9m 31s (- 22m 13s) (45000 30%) 2.2012
10m 34s (- 21m 8s) (50000 33%) 2.1370
11m 37s (- 20m 4s) (55000 36%) 2.1030
12m 39s (- 18m 59s) (60000 40%) 2.0296
13m 42s (- 17m 55s) (65000 43%) 1.9408
14m 45s (- 16m 52s) (70000 46%) 1.9263
15m 48s (- 15m 48s) (75000 50%) 1.8574
16m 51s (- 14m 45s) (80000 53%) 1.7943
17m 54s (- 13m 41s) (85000 56%) 1.7575
18m 57s (- 12m 38s) (90000 60%) 1.7528
20m 0s (- 11m 35s) (95000 63%) 1.6581
21m 3s (- 10m 31s) (100000 66%) 1.6490
22m 6s (- 9m 28s) (105000 70%) 1.6144
23m 10s (- 8m 25s) (110000 73%) 1.6010
24m 13s (- 7m 22s) (115000 76%) 1.5472
25m 16s (- 6m 19s) (120000 80%) 1.5584
26m 19s (- 5m 15s) (125000 83%) 1.5142
27m 22s (- 4m 12s) (130000 86%) 1.5065
28m

In [ ]:
evaluateRandomly(encoder1, decoder1)

> ty deistvitel no zdorovo vygliadish .
= you really do look great .
< you really are look beautiful . <EOS>

> mne skazali chto tom uekhal za granitsu .
= i was told that tom went abroad .
< i was told tom was interested in . <EOS>

> kak skoro ty mozhesh zakonchit eti risunki ?
= how quickly can you finish these pictures ?
< how soon can you be such my hands ? <EOS>

> u tebia eshchio est voprosy ?
= do you still have questions ?
< do you have any more ? <EOS>

> khotite chtoby ia ostalsia zdes ?
= do you want me to stay here ?
< do you want me to stay here ? <EOS>

> tebe ponravilos ili net ?
= did you like that or not ?
< did you like that or not ? <EOS>

> kak chasto ty pol zuesh sia kameroi na svoiom smartfone ?
= how often do you use the camera on your smartphone ?
< how often do you use your your ? <EOS>

> ty znala chto u tebia krasivye glaza ?
= did you know you have pretty eyes ?
< did you know that you you you ? ? ? <EOS>

> ia gotov sdelat chto ugodno chtoby pomoch tebe .


## Мы видим, что качество перевода с использованием 2х-слойных GRU для энкодера/декодера несущественно улучшилось (лосс=1.3766 на 2х-слойных против лосса=1.4227 на 1-слойных). Время обучения модели при этом практически не увеличилось.

## Попробуем теперь заменить 2х-слойную GRU на 1-слойную LSTM.

In [10]:
# class EncoderLSTM_test(nn.Module):
#     def init(self, inputsize, hiddensize):
#       super(EncoderLSTM_test, self).init()
#       self.hiddensize = hiddensize

#       self.embedding = nn.Embedding(inputsize, hidden_size)
#       self.lstm = nn.LSTM(hidden_size, hidden_size)

#     def forward(self, input, hidden):
#       embedded = self.embedding(input).view(1, 1, -1)
#       output = embedded
#       output, hidden = self.lstm(output, hidden)
#       return output, hidden

#     def initHidden(self):
#       return torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size)

In [24]:
class EncoderLSTM_new(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM_new, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size).to(device), torch.zeros(1, 1, self.hidden_size).to(device)

In [25]:
class DecoderLSTM_new(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM_new, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size).to(device), torch.zeros(1, 1, self.hidden_size).to(device)

In [26]:
hidden_size = 256
encoder1 = EncoderLSTM_new(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderLSTM_new(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 150000, print_every=5000) 

1m 4s (- 31m 2s) (5000 3%) 3.8467
1m 59s (- 27m 55s) (10000 6%) 3.4887
2m 54s (- 26m 8s) (15000 10%) 3.1790
3m 49s (- 24m 52s) (20000 13%) 2.9905
4m 44s (- 23m 40s) (25000 16%) 2.8287
5m 42s (- 22m 51s) (30000 20%) 2.7193
6m 38s (- 21m 48s) (35000 23%) 2.6156
7m 33s (- 20m 46s) (40000 26%) 2.5188
8m 28s (- 19m 46s) (45000 30%) 2.4546
9m 24s (- 18m 48s) (50000 33%) 2.3970
10m 18s (- 17m 48s) (55000 36%) 2.3431
11m 13s (- 16m 50s) (60000 40%) 2.2334
12m 8s (- 15m 53s) (65000 43%) 2.2143
13m 3s (- 14m 55s) (70000 46%) 2.1594
14m 3s (- 14m 3s) (75000 50%) 2.0957
14m 58s (- 13m 6s) (80000 53%) 2.0278
15m 54s (- 12m 9s) (85000 56%) 1.9983
16m 49s (- 11m 13s) (90000 60%) 1.9601
17m 45s (- 10m 17s) (95000 63%) 1.9030
18m 41s (- 9m 20s) (100000 66%) 1.8662
19m 36s (- 8m 24s) (105000 70%) 1.8393
20m 31s (- 7m 27s) (110000 73%) 1.7972
21m 33s (- 6m 33s) (115000 76%) 1.7636
22m 29s (- 5m 37s) (120000 80%) 1.7351
23m 24s (- 4m 40s) (125000 83%) 1.7025
24m 19s (- 3m 44s) (130000 86%) 1.6714
25m 14s 

In [29]:
evaluateRandomly(encoder1, decoder1)

> kak ia vygliazhu ?
= how do i look ?
< how do i get ? <EOS>

> tom ne zabyl sdelat to o chiom my ego prosili ?
= did tom remember to do what we asked him to do ?
< did tom want to do what what do asked to do ? <EOS>

> mne net ravnykh .
= i have no equal .
< i have no . . <EOS>

> ia byl slishkom slab chtoby vstat s posteli .
= i was too weak to get out of bed .
< i was too tired to get out of bed . <EOS>

> on deistvitel no dzhentl men .
= he is quite a gentleman .
< he is really that . <EOS>

> ia nadeialsia chto smogu pomoch tomu eto sdelat .
= i was hoping i could help tom do that .
< i was hoping we could help do that . <EOS>

> ty eto pomnish ?
= do you remember that ?
< do you remember this ? <EOS>

> kak eto ty tak zdorovo poranilsia ?
= how did you get hurt so badly ?
< how did you get so so so ? <EOS>

> ty obiazan eto delat ?
= are you obliged to do that ?
< did you intend to do that ? <EOS>

> klianiotes li vy govorit pravdu i nichego krome pravdy ?
= do you swear to tell

## Мы видим, что на 1-слойной LSTM качество перевода ухудшилось как визуально, так и на основании лосса (1.5499). Время обучения (28 мин.) сопоставимо с GRU.

## Наилучшее качество было на 2х-слойной GRU (лосс 1.3766), чуть хуже - на 1-слойной GRU (лосс 1.4227). 

## Полагаю, что на LSTM удалось бы существенно улучшить качество, добавив ещё слой LSTM и сделав сеть двунаправленной.